Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, Run, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice, Webservice
from azureml.core.model import Model

## test services
import requests
import json

In [ ]:
ws = Workspace.from_config()
#TODO: load variables from config file

env_name = "AzureML-Minimal"

project_folder = '../src'
script_name = 'score.py'
model_name = 'my_model.pkl'
service_name = 'myservice'

In [ ]:
# env = Environment("LocalDeploy")
env = Environment.get(ws, env_name)

#add env name to retrieve model
env.environment_variables['MODEL_NAME'] = model_name

#TODO: retrieve from local env file
for pip_package in ["pip","scikit-learn","joblib"]:
    env.python.conda_dependencies.add_pip_package(pip_package)
# env.python.conda_dependencies.add_pip_package('azureml-sdk')

## Local deployment for debugging
deployment_config = LocalWebservice.deploy_configuration(port=6789)
## ACI deployment for DEV
# deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

inference_config = InferenceConfig(entry_script=project_folder+'/'+script_name, environment=env)

#link to the registered model
model = Model(workspace=ws,name=model_name)

In [ ]:
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)


In [ ]:
print(service.state)

In [ ]:
print(service.get_logs())

In [ ]:
## If you need to get the service from azure or local
# service = Webservice(ws,service_name)
# service = LocalWebservice(ws,service_name)


In [ ]:
# If you need to update the service
# service.reload()


In [ ]:
data = {
    'data':
    {
        'WeekStarting': {
            0: '1990-06-14 00:00:00',1: '1990-06-21 00:00:00',
            2: '1990-06-28 00:00:00',
            3: '1990-07-05 00:00:00',
            4: '1990-07-12 00:00:00'
            },
        'Store': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000},
        'Brand': {0: 'dominicks',1: 'dominicks',2: 'dominicks',3: 'dominicks',4: 'dominicks'},
        'Quantity': {0: 12003, 1: 10239, 2: 17917, 3: 14218, 4: 15925},
        'Advert': {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
        'Price': {0: 2.59, 1: 2.39, 2: 2.48, 3: 2.33, 4: 2.01},
        'Revenue': {0: 31087.769999999997,1: 24471.210000000003,2: 44434.159999999996,3: 33127.94,4: 32009.249999999996}
        }
  }


input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# URL for the web service
scoring_uri = "http://localhost:6789/score"
    

if not isinstance(service,LocalWebservice):

    #set uri
    scoring_uri = service.scoring_uri

    # If the service is authenticated, set the key or token
    key = service.swagger_uri

    # If authentication is enabled, set the authorization header
    headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
# resp = service.run(input_data)
print(resp.text)

In [ ]:
# Delete webservice when done
# service.delete()